In [2]:
%pip install opencv-python tensorflow pillow scikit-learn

import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


Note: you may need to restart the kernel to use updated packages.


2024-10-28 22:15:46.395577: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-28 22:15:46.397009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 22:15:46.401212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 22:15:46.414068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 22:15:46.435551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:

# Caminhos
data_dir = './faces'  # Pasta com os dados de treino
output_dir = './faces_detectadas'

# Parâmetros
img_width, img_height = 150, 150
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Cria o diretório de saída se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Inicializa contador de imagem
img_counter = 0


In [ ]:

# Varre cada subpasta dentro do diretório de dados
for person_name in os.listdir(data_dir):
    person_path = os.path.join(data_dir, person_name)

    # Verifica se é um diretório (para garantir que é uma pasta de pessoa)
    if os.path.isdir(person_path):
        # Itera sobre cada imagem dentro da subpasta da pessoa
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)

            if img is not None:
                # Converte a imagem para escala de cinza (requisito para o Haar Cascade)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Detecta faces na imagem
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                # Para cada face detectada, recorta e salva com o nome formatado
                for (x, y, w, h) in faces:
                    face_img = img[y:y+h, x:x+w]
                    resized_face = cv2.resize(face_img, (img_width, img_height))

                    # Define o nome do arquivo de saída
                    output_filename = f'img{img_counter}_{person_name}.jpg'
                    output_path = os.path.join(output_dir, output_filename)

                    # Salva a face detectada
                    cv2.imwrite(output_path, resized_face)
                    print(f'Face detectada salva em: {output_path}')

                    # Incrementa o contador de imagens
                    img_counter += 1
            else:
                print(f'Erro ao carregar imagem: {img_path}')




In [5]:
# Atualiza o diretório de dados para as faces detectadas
data_dir_faces = './faces_detectadas'

# Data augmentation e preparação do gerador de dados de treino para as faces detectadas
datagen_faces = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator_faces = datagen_faces.flow_from_directory(
    data_dir_faces,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator_faces = datagen_faces.flow_from_directory(
    data_dir_faces,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Impressão de informações do gerador
print("Classes detectadas: ", train_generator_faces.class_indices)
print("Número de imagens de treinamento: ", train_generator_faces.samples)
print("Número de imagens de validação: ", validation_generator_faces.samples)

NameError: name 'batch_size' is not defined

In [52]:
# Caminhos e Parâmetros
data_dir = './faces_detectadas'
img_width, img_height = 150, 150
batch_size = 10

# Carregar e organizar os dados manualmente
images = []
labels = []

for filename in os.listdir(data_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Carrega a imagem e redimensiona
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_width, img_height))
        
        # Normaliza a imagem
        img = img / 255.0
        images.append(img)
        
        # Extrai o rótulo do nome do arquivo (assumindo o formato 'img{numero}_{nome}.jpg')
        label = filename.split('_')[1].split('.')[0]
        labels.append(label)

# Converte as listas em arrays numpy
images = np.array(images)
labels = np.array(labels)

# Codificação das Labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
labels_categorical = to_categorical(labels_encoded, num_classes)

# Gerador de dados para aumento de dados
data_gen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
train_generator = data_gen.flow(images, labels_categorical, batch_size=batch_size)

# Definição do Modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento do Modelo
model.fit(train_generator, epochs=40)

# Salvar o Modelo Treinado
model.save('face_recognition_model.h5')

print("Modelo treinado e salvo com sucesso.")

/home/vinicius/Documents/Aulas/ProcessamentoImagem/face_detection/face_detection/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/vinicius/Documents/Aulas/ProcessamentoImagem/face_detection/face_detection/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 382ms/step - accuracy: 0.1488 - loss: 2.8602
Epoch 2/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 302ms/step - accuracy: 0.2279 - loss: 2.1583
Epoch 3/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step - accuracy: 0.2420 - loss: 1.6546
Epoch 4/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 288ms/step - accuracy: 0.0541 - loss: 1.6006
Epoch 5/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.2337 - loss: 1.6051
Epoch 6/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step - accuracy: 0.3954 - loss: 1.5784
Epoch 7/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - accuracy: 0.3476 - loss: 1.5873
Epoch 8/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step - accuracy: 0.3990 - loss: 1.5712
Epoch 9/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step - accuracy: 0.4517 - loss: 1.5196
Epoch 10/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.2904 - loss: 1.5041
Epoch 11/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step - accuracy: 0.2279 - loss: 1.5383
Epoch 12/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 306ms/step - accuracy: 0.5990 - lo

Modelo treinado e salvo com sucesso.


In [5]:
# Caminhos
test_dir = './faces/lebron'  # Diretório das imagens de teste
model_path = './face_recognition_model.h5'

# Parâmetros da imagem
img_width, img_height = 150, 150

# Carrega o modelo treinado
model = load_model(model_path)

# Recria o LabelEncoder usado no treinamento
labels = [filename.split('_')[1].split('.')[0] for filename in os.listdir('./faces_detectadas')]
label_encoder = LabelEncoder()
label_encoder.fit(labels)
num_classes = len(label_encoder.classes_)

# Listas para armazenar rótulos reais e predições
y_true = []
y_pred = []


# Laço para processar cada imagem de teste
for filename in os.listdir(test_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Caminho completo da imagem
        img_path = os.path.join(test_dir, filename)
        
        # Carrega a imagem original para detecção de rosto
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converte para RGB para processamento
        faces = face_cascade.detectMultiScale(img_rgb, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Laço para cada rosto detectado
        for (x, y, w, h) in faces:
            # Extrai e redimensiona a face detectada
            face = img_rgb[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (img_width, img_height)) / 255.0
            face_resized = np.expand_dims(face_resized, axis=0)
            
            # Realiza a predição
            prediction = model.predict(face_resized)
            predicted_class = np.argmax(prediction, axis=1)
            predicted_label = label_encoder.inverse_transform(predicted_class)[0]
            
            # Desenha o retângulo e o rótulo na imagem
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Salva a imagem rotulada
        # output_path = os.path.join(output_dir, f"labeled_{filename}")
        # cv2.imwrite(output_path, img)
        cv2.imshow(filename, img)
        cv2.waitKey(990)  # Aguarda uma tecla para exibir a próxima imagem
        cv2.destroyAllWindows()  
        # print(f"Imagem {filename} processada e salva em {output_path}")

# Converte os rótulos reais para inteiros correspondentes
y_true_encoded = label_encoder.transform(y_true)
y_pred_encoded = label_encoder.transform(y_pred)

# Calcula a acurácia
accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
print(f"Acurácia: {accuracy:.2f}")

# Calcula o F1-Score
f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted')
print(f"F1-Score (Weighted): {f1:.2f}")

# Calcula e exibe a Matriz de Confusão
conf_matrix = confusion_matrix(y_true_encoded, y_pred_encoded)
print("Matriz de Confusão:")
print(conf_matrix)

# Relatório de Classificação Completo (opcional)
print("\nRelatório de Classificação:")
print(classification_report(y_true_encoded, y_pred_encoded, target_names=label_encoder.classes_))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Acurácia: nan
F1-Score (Weighted): nan
Matriz de Confusão:
[]

Relatório de Classificação:


/home/vinicius/Documents/Aulas/ProcessamentoImagem/face_detection/face_detection/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/vinicius/Documents/Aulas/ProcessamentoImagem/face_detection/face_detection/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: Number of classes, 0, does not match size of target_names, 31. Try specifying the labels parameter

In [35]:
# # Carrega o modelo treinado
# model = load_model('./face_recognition_model.h5')

# # Dicionário de classes
# class_dict = {v: k for k, v in train_generator.class_indices.items()}
# df = cv2.CascadeClassifier('classifier.xml')

# input_dir = './faces/elvis'

# # Processa cada imagem na pasta 'faces'
# for filename in os.listdir(input_dir):
#     if filename.endswith('.jpg'):
#         file_path = os.path.join(input_dir, filename)
#         i = cv2.imread(file_path)
#         iPB = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)

#         # Executa a detecção de faces
#         faces = df.detectMultiScale(iPB, scaleFactor=1.05, minNeighbors=7, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)

#         # Salva cada face detectada na nova pasta com rótulo
#         for j, (x, y, w, h) in enumerate(faces):
#             face = i[y:y+h, x:x+w]
#             face_resized = cv2.resize(face, (img_width, img_height))
#             face_normalized = face_resized / 255.0
#             face_reshaped = np.reshape(face_normalized, (1, img_width, img_height, 3))
#             prediction = model.predict(face_reshaped)
#             label = class_dict[np.argmax(prediction)]
#             face_filename = os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_{label}_{j}.jpg')
#             cv2.imwrite(face_filename, face)
#             cv2.rectangle(i, (x, y), (x + w, y + h), (0, 255, 255), 7)
#             cv2.putText(i, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

#         # Exibe a imagem com as faces detectadas e rotuladas
#         cv2.imshow(f'{filename} - {len(faces)} face(s) encontrada(s)', i)
#         cv2.waitKey(9900)
#         cv2.destroyAllWindows()
